In [1]:
from secret import openai_key
import os

os.environ['OPENAI_API_KEY'] = openai_key

In [5]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.6)
#llm("I am a R&D sciantists. My objective is to support my company in discovery of new drugs. I run patent analysis. Answer this question based on the patent information.")
llm = 

'\n\nFancy Name: Schloss Bierhaus \nFavorite Meal: Wiener Schnitzel (breaded and fried veal cutlet) with Spätzle (German egg noodles) and a side of Sauerkraut.'

In [7]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this.")

prompt_template_name.format(cuisine="Italian")

'I want to open a restaurant for Italian food. Suggest a fency name for this.'

In [9]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.run("American")

'\n\n"The All-American Table"'

In [ ]:
from secret import openai_key
import os
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.vectorstores import ElasticVectorSearch
from secret import elasticsearch_username, elasticsearch_password
from langchain.embeddings import HuggingFaceEmbeddings

os.environ['OPENAI_API_KEY'] = openai_key

llm = OpenAI(temperature=0.6)

def make_the_llm():
   template_informed = """
   I am a helpful AI that answers questions.
   When I don't know the answer I say I don't know.
   I know context: {context}
   when asked: {question}
   my response using only information in the context is: """
   prompt_informed = PromptTemplate(
       template=template_informed,
       input_variables=["context", "question"])
   return LLMChain(prompt=prompt_informed, llm=llm)

# Create the HuggingFace Transformer like before
model_name = "sentence-transformers/all-distilroberta-v1"
hf = HuggingFaceEmbeddings(model_name=model_name)

topic = "patents"

# Connect with Elasticsearch
es_endpoint = "patchat.es.europe-west3.gcp.cloud.es.io"
es_username = elasticsearch_username
es_password = elasticsearch_password
es_index_name = "patents-embeddings-1"

es_url = f"https://{es_username}:{es_password}@{es_endpoint}:9243"
es_db = ElasticVectorSearch(embedding=hf, elasticsearch_url=es_url, index_name=es_index_name)

## set up the conversational LLM
llm_chain_informed = make_the_llm()

def ask_a_question(question):
   ## get the relevant chunk from Elasticsearch for a question
   similar_docs = es_db.similarity_search(question)
   print(f'The most relevant passage: \n\t{similar_docs[0].page_content}')
   informed_context= similar_docs[0].page_content
   informed_response = llm_chain_informed.run(
       context=informed_context,
       question=question)
   return informed_response


# The conversational loop
print(f'I am a trivia chat bot, ask me any question about {topic}')
while True:
   command = input("User Question >> ")
   response= ask_a_question(command)
   print(f"\tAnswer  : {response}")


I am a trivia chat bot, ask me any question about patents
User Question >> list innovations related to information processing.
The most relevant passage: 
	Patent title: Information processing method, garment order reception management apparatus, and non-transitory computer-readable storage medium.\nPatent abstract: An information processing method performed by a processor, the method comprises garment selecting of receiving a selection of a garment from a user, price calculating of calculating a price of the garment on the basis of a type of the selected garment and parameters relating to a design and materials set for the garment, display controlling of causing a display part to display the calculated price, and parameter changing of changing the parameters in conjunction with an input from the user. The display controlling includes causing the display part to display the price that reflects the changed parameters in response to the change of the parameters.\n\nPatent publication num

	Answer  : 

Information processing, garment, design, materials, parameters, manufacturing process, unit price, budget, blending ratio, storage medium, computer.
User Question >> What problem does inventions related to communication solve?
The most relevant passage: 
	Patent title: Information processing method, garment order reception management apparatus, and non-transitory computer-readable storage medium.\nPatent abstract: An information processing method performed by a processor, the method comprises garment selecting of receiving a selection of a garment from a user, price calculating of calculating a price of the garment on the basis of a type of the selected garment and parameters relating to a design and materials set for the garment, display controlling of causing a display part to display the calculated price, and parameter changing of changing the parameters in conjunction with an input from the user. The display controlling includes causing the display part to display the 

	Answer  : 
   Information processing, garment, parameters, design, materials, price, display, budget, blending ratio, property, storage medium.


In [ ]:
from secret import openai_key
import os
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.vectorstores import ElasticVectorSearch
from secret import elasticsearch_username, elasticsearch_password
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import SequentialChain
import json

os.environ['OPENAI_API_KEY'] = openai_key
llm = OpenAI(temperature=0.6)

# Create the HuggingFace Transformer like before
model_name = "sentence-transformers/all-distilroberta-v1"
hf = HuggingFaceEmbeddings(model_name=model_name)

# Connect with Elasticsearch
es_endpoint = "patchat.es.europe-west3.gcp.cloud.es.io"
es_username = elasticsearch_username
es_password = elasticsearch_password
es_index_name = "patents-embeddings-1"

es_url = f"https://{es_username}:{es_password}@{es_endpoint}:9243"
es_db = ElasticVectorSearch(embedding=hf, elasticsearch_url=es_url, index_name=es_index_name)

def step_1_identify_key_concepts_chain():
    key_concepts_question = """
       You are a scientist or patent layer.
       It is your job to find prior art in patents.
       The following context describes patents consisting of title, abstract, publication number and some other information: {context}
       Identify the key concepts of the patent.
       When providing an answer, put the publication number followed by the title first.
       Ignore any citation publication numbers.
       When asked: {question}
       Your answer to the questions using only information in the context is: """
    key_concepts_prompt = PromptTemplate(template=key_concepts_question, input_variables=["context", "question"])
    return LLMChain(prompt=key_concepts_prompt, llm=llm, output_key="key_concepts")

def step_2_identify_keywords_chain():
    keywords_question = """
        Given the key concepts of a patent, generate keywords for further prior art research.
        Use synonyms and related keywords based on your knowledge.
        Here are the identified key concepts: {key_concepts}
    """
    keywords_prompt = PromptTemplate(input_variables = ['key_concepts'], template=keywords_question)
    return LLMChain(prompt=keywords_prompt, llm=llm, output_key="keywords")

def step_3_determine_ipc_symbols():
    # use previously generated keywords to identify patents and aggregate their IPC sympbols
    return "H01L 31/00"

chain = SequentialChain(
        chains = [
            step_1_identify_key_concepts_chain(),
            step_2_identify_keywords_chain()
        ],
        input_variables = ['question', 'context'],
        output_variables = ['key_concepts', "keywords"],
        verbose=False
    )

def ask_question(question):
   ## get the relevant chunk from Elasticsearch for a question
   similar_docs = es_db.similarity_search(question)
   #print(f'The most relevant passage: \n\t{similar_docs[0].page_content}')
   context= similar_docs[0].page_content
   #return chain.r(context=context, question=question, output_key = ['keywords'])
   return chain({"question" : question, "context" : context})


# The conversational loop
while True:
   question = input("User Question >> ")
   response = ask_question(question)
   del response['question']
   del response['context']
   response = json.dumps(response)
   print(f"\tAnswer  : {response}")



User Question >> list innovations related to communication methods
	Answer  : {"key_concepts": "\n       The information processing method includes receiving a selection of a garment from a user, calculating a price of the garment on the basis of a type of the selected garment and parameters relating to a design and materials set for the garment, causing a display part to display the calculated price, and changing the parameters in conjunction with an input from the user. The display controlling includes causing the display part to display the price that reflects the changed parameters in response to the change of the parameters. The information processing method further includes generating an appearance image of the garment on the basis of the parameters, wherein the display controlling includes causing the display part to display the appearance image that reflects the changed parameters in response to an input from the user. Additionally, the price calculating includes calculating a 